In [70]:
### Feature Engineering ###
# one hot encoding
# new features
# ...

In [94]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [95]:
# load the etl data from cos

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3b08f636aa4c4bc58b03e36105622462 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='EI06iPS11MRoHQiLAPwMFIukCpW_Yx5PqbVhI0FSYqjZ',
    ibm_auth_endpoint="https://iam.eu-de.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_3b08f636aa4c4bc58b03e36105622462.get_object(Bucket='courserabadges-donotdelete-pr-72y9f6bzbckngj',Key='online_shoppers_intention_etl.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,0
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,0
2,0.0,0.0,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,0
3,0.0,0.0,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,0
4,0.0,0.0,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,Feb,2,2,1,3,Returning_Visitor,False,0


In [96]:
df.shape

(12283, 18)

In [97]:
# one-hot-encoding
def encode_one_hot(df, columns, replace_original=False):
    feature_sets = list()
    for col in columns:
        if col not in list(df.columns):
            print(col, '--- Column does not exist in DataFrame Object')
            return(None)
        df_temp = df[[col]]
        enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
        raw_features = enc.fit_transform(df_temp)
        raw_col_names = enc.get_feature_names()
        col_names = [str(col + '_' + name) for name in raw_col_names]
        features = pd.DataFrame(raw_features, columns=col_names)
        print(col, '--- shape: ', raw_features.shape)
        feature_sets.append(features)
        
    merged_features = pd.concat(feature_sets, axis=1)
    if replace_original is True:
        df = df.drop(columns=columns)
        df = pd.concat([merged_features, df], axis=1)
        return df
    else:
        return merged_features

In [98]:
columns_to_encode = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType']
df = encode_one_hot(df=df, columns=columns_to_encode, replace_original=True)

Month --- shape:  (12283, 10)
OperatingSystems --- shape:  (12283, 8)
Browser --- shape:  (12283, 13)
Region --- shape:  (12283, 9)
TrafficType --- shape:  (12283, 20)
VisitorType --- shape:  (12283, 3)


In [99]:
# binary feature Weekend
df['Weekend'] = df['Weekend'].apply(lambda x: 1 if (x is True) else 0)
df.shape

(12283, 75)

In [100]:
df.head()

,Month_x0_Aug,Month_x0_Dec,Month_x0_Feb,Month_x0_Jul,Month_x0_June,Month_x0_Mar,Month_x0_May,Month_x0_Nov,Month_x0_Oct,Month_x0_Sep,...,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Weekend,Revenue
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,0,0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.0,0.0,0,0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.0,0.0,1,0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,0,0


In [101]:
# transform (e.g. sqrt or log of the durations) certain features and or take out outliers
def feature_transform(df, columns, func, add):
    df_temp = df[columns].astype('float64')
    df_transformed = pd.DataFrame(df_temp.apply(lambda x: func(x+add)), columns = df_temp.columns)
    df = df.drop(columns=columns)
    df = pd.concat([df_transformed, df], axis=1)
    return df

In [102]:
def scale_features(df, columns):
    df_temp = df[columns].astype('float64')
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(X=df_temp), columns = df_temp.columns)
    df = df.drop(columns=columns)
    df = pd.concat([df_scaled, df], axis=1)
    return df

In [103]:
cols_scaling = ['Administrative',
             'Administrative_Duration',
             'Informational',
             'Informational_Duration',
             'ProductRelated',
             'ProductRelated_Duration',
             'BounceRates',
             'ExitRates',
             'PageValues',
             'SpecialDay']

In [104]:
# store file in COS as CSV

# option 1: plain
df_1 = scale_features(df=df, columns=cols_scaling)
csv_body = df_1.to_csv(sep=',', encoding='UTF-8', index=False)
meta_data = client_3b08f636aa4c4bc58b03e36105622462.put_object(Body=csv_body, Bucket='courserabadges-donotdelete-pr-72y9f6bzbckngj',Key='online_shoppers_intention_eng.csv')

# option2: sqrt transformation
# df = feature_transform(df=df, columns=cols_scaling, func=np.log, add=1)
# df_2 = scale_features(df=df, columns=cols_scaling)
csv_body = df_2.to_csv(sep=',', encoding='UTF-8', index=False)
meta_data = client_3b08f636aa4c4bc58b03e36105622462.put_object(Body=csv_body, Bucket='courserabadges-donotdelete-pr-72y9f6bzbckngj',Key='online_shoppers_intention_eng_tf.csv')



In [106]:
1

1